In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio, guardar_df_en_minio, generar_ruta_fecha
import os
from dotenv import load_dotenv

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']
Conexión exitosa a PostgreSQL PostgreSQL 16.0, compiled by Visual C++ build 1935, 64-bit
2025/05/06


In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [ ]:
def extract_pdf_content(pdf_document,examen):
    """
    Extrae contenido específico de un archivo PDF ya abierto.

    Args:
        pdf_document (fitz.Document): Documento PDF abierto con PyMuPDF.

    Returns:
        dict: Diccionario con la información extraída.
    """
    extracted_data = {
        #"test":None,
        "Folio": None,
        "Fecha Impresion": None,
        "Identificacion": None,
        "Numero de carpeta": None,
        "Nombre del paciente": None,
        "Edad Actual": None,
        "Sexo": None,
        "Ingreso": None,
        "Fecha de Ingreso": None,
        "Fecha Evolucion:": None,
        "Cama": None,
        "Servicio": None,
        "Entidad": None,
        "Fecha de Toma": None,
        "Fecha de Recepcion": None,
        "Fecha de Informe": None,
        "Informe de Patologia": None,
        "Material Enviado": None,
        "Descripcion Macroscopica": None,
        "Descripcion Microscopica": None,
        "Diagnostico Histopatologico": None,
        "Medico Patologo:": None,
        "Registro Profesional": None,
        "Nombre reporte": None,
        "Usuario": None
    }
    
    try:
        # Leer cada página del PDF
        for page_number in examen["paginas"]:#range(pdf_document.page_count):
            page = pdf_document[page_number]
            text = page.get_text("text")
            cleaned_text = " ".join(text.split())
            
            # Extraer información utilizando búsquedas            
            if "FOLIO" in cleaned_text:
                extracted_data["Folio"] = cleaned_text.split("FOLIO")[1].split()[0].strip()
            if "FECHA DE IMPRESION" in cleaned_text:
                extracted_data["Fecha Impresion"] = cleaned_text.split("FECHA DE IMPRESION")[1].split("HOJA")[0].strip()
            if "Identificación:" in cleaned_text:
                extracted_data["Identificacion"] = cleaned_text.split("Identificación:")[1].split("Numero de carpeta:")[0].strip()
            if "Numero de carpeta:" in cleaned_text:
                extracted_data["Numero de carpeta"] = cleaned_text.split("Numero de carpeta:")[1].split("Nombre del paciente:")[0].strip()
            if "Nombre del paciente:" in cleaned_text:
                extracted_data["Nombre del paciente"] = cleaned_text.split("Nombre del paciente:")[1].split("Edad Actual")[0].strip()
            if "Edad Actual" in cleaned_text:
                extracted_data["Edad Actual"] = cleaned_text.split("Edad Actual")[1].split("Sexo")[0].strip()
            if "Sexo" in cleaned_text:
                extracted_data["Sexo"] = cleaned_text.split("Sexo")[1].split("DATOS DE LA ATENCION")[0].strip()
            if "Ingreso " in cleaned_text:
                extracted_data["Ingreso"] = cleaned_text.split("Ingreso ")[1].split(" Fecha")[0].strip()
            if "Fecha Ingreso" in cleaned_text:
                extracted_data["Fecha de Ingreso"] = cleaned_text.split("Fecha Ingreso")[1].split("Fecha Evolucion:")[0].strip()
            if "Fecha Evolucion:" in cleaned_text:
                extracted_data["Fecha Evolucion"] = cleaned_text.split("Fecha Evolucion:")[1].split("Cama")[0].strip()
            if "Cama" in cleaned_text:
                extracted_data["Cama"] = cleaned_text.split("Cama")[1].split("Servicio")[0].strip()
            if "Servicio" in cleaned_text:
                extracted_data["Servicio"] = cleaned_text.split("Servicio")[1].split("ENTIDAD")[0].strip()
            if "ENTIDAD" in cleaned_text:
                extracted_data["Entidad"] = cleaned_text.split("ENTIDAD")[1].split("INFORME DE PATOLOGIA")[0].strip()
            if "FECHA DE TOMA:" in cleaned_text:
                extracted_data["Fecha de Toma"] = cleaned_text.split("FECHA DE TOMA:")[1].split("FECHA DE RECEPCION:")[0].strip()
            if "FECHA DE RECEPCION:" in cleaned_text:
                extracted_data["Fecha de Recepcion"] = cleaned_text.split("FECHA DE RECEPCION:")[1].split("FECHA DE INFORME :")[0].strip()
            if "FECHA DE INFORME :" in cleaned_text:
                #extracted_data["Fecha de Informe"] = cleaned_text.split("FECHA DE INFORME :")[1].split("INFORME DE PATOLOGIA No.")[0].strip()
                txt = cleaned_text.split("FECHA DE INFORME :")[1]
                # Usar regex para capturar hasta encontrar la primera letra (mayúscula o minúscula)
                match = re.match(r'([^A-Za-z]*)', txt)
                if match:
                    extracted_data["Fecha de Informe"] = match.group(1).strip()                
            if "INFORME DE PATOLOGIA No." in cleaned_text:
                extracted_data["Informe de Patologia"] = cleaned_text.split("INFORME DE PATOLOGIA No.")[1].split("MATERIAL ENVIADO :")[0].strip()
            #Material Enviado
            #if "MATERIAL ENVIADO :" in cleaned_text:
            #    extracted_data["Material Enviado"] = cleaned_text.split("MATERIAL ENVIADO :")[1].split("DESCRIPCION MACROSCOPICA")[0].strip()
            if "FECHA DE INFORME :" in cleaned_text:                
                txt = cleaned_text.split("FECHA DE INFORME :")[1].split("INFORME DE PATOLOGIA No.")[0].strip()
                match = re.search(r'\d{2}/\d{2}/\d{4}\s+\d{1,2}:\d{2}\s+(.+)', txt)
                if match:
                    extracted_data["Material Enviado"] = match.group(1).strip()            
            if "DESCRIPCION MACROSCOPICA" in cleaned_text:
                extracted_data["Descripcion Macroscopica"] = cleaned_text.split("DESCRIPCION MACROSCOPICA")[1].split("DESCRIPCION MICROSCOPICA")[0].strip()
            if "DESCRIPCION MICROSCOPICA" in cleaned_text:
                extracted_data["Descripcion Microscopica"] = cleaned_text.split("DESCRIPCION MICROSCOPICA")[1].split("DIAGNOSTICO HISTOPATOLOGICO")[0].strip()
            if "DIAGNOSTICO HISTOPATOLOGICO" in cleaned_text:                
                extracted_data["Diagnostico Histopatologico"] = cleaned_text.split("DIAGNOSTICO HISTOPATOLOGICO")[1].split("Medico Patologo:")[0].strip()
            if "Medico Patologo:" in cleaned_text:                
                extracted_data["Medico Patologo"] = cleaned_text.split("Medico Patologo:")[1].split("Reg. Prof.:")[0].strip()
            if "Reg. Prof.:" in cleaned_text:
                extracted_data["Registro Profesional"] = cleaned_text.split("Reg. Prof.:")[1].split("Nombre reporte :")[0].strip()
            if "Nombre reporte :" in cleaned_text:
                extracted_data["Nombre reporte"] = cleaned_text.split("Nombre reporte :")[1].split("Usuario:")[0].strip()
            if "Usuario:" in cleaned_text:
                extracted_data["Usuario"] = cleaned_text.split("Usuario:")[1].split("LICENCIADO A:")[0].strip()

    except Exception as e:
        print(f"Error al extraer datos del PDF: {e}")

    return extracted_data

In [ ]:
def funcion_que_separa_cada_examen_por_archivo(pdf_document):
    """
    Separa un PDF en una lista de exámenes, donde cada examen es un diccionario con 'folio' y 'paginas' (lista de números de página).
    
    Args:
        pdf_document (fitz.Document): Documento PDF abierto.
    
    Returns:
        list: Lista de diccionarios, cada uno representando un examen con su folio y páginas asociadas.
    """
    examenes = []
    folio_actual = None
    paginas_examen = []
    
    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]
        text = page.get_text("text")
        cleaned_text = " ".join(text.split())
        
        # Extraer folio de la página
        folio = None
        if "FOLIO" in cleaned_text:
            folio = cleaned_text.split("FOLIO")[1].split()[0].strip()
        
        if folio:
            if folio != folio_actual:
                # Guardar el examen anterior si existe
                if folio_actual and paginas_examen:
                    examenes.append({"folio": folio_actual, "paginas": paginas_examen})
                # Iniciar nuevo examen
                folio_actual = folio
                paginas_examen = [page_number]
            else:
                # Agregar página al examen actual
                paginas_examen.append(page_number)
    
    # Agregar el último examen
    if folio_actual and paginas_examen:
        examenes.append({"folio": folio_actual, "paginas": paginas_examen})
    
    print(examenes)    
    return examenes


In [ ]:
import pandas as pd
import fitz  # PyMuPDF

def process_all_pdfs_in_minio(minio_client, bucket_name, folder_path):
    """
    Procesa todos los archivos PDF en una ruta específica de un bucket MinIO.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde están los archivos
    folder_path : str
        Ruta dentro del bucket (ej: 'informes/2023/')
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los datos extraídos de todos los PDFs
    """
    results = []
    
    try:
        # Asegurar que la ruta termine con '/'
        if not folder_path.endswith('/'):
            folder_path += '/'
        
        # Listar objetos en la ruta especificada
        objects = minio_client.list_objects(bucket_name, prefix=folder_path, recursive=True)
        
        #cada obj es un archivo
        for obj in objects:
            if obj.object_name.lower().endswith('.pdf'):
                print(f"Procesando archivo: {obj.object_name}")
                pdf_data = None
                pdf_document = None
                
                try:
                    # Obtener el objeto PDF
                    response = minio_client.get_object(bucket_name, obj.object_name)
                    pdf_data = BytesIO(response.read())
                    
                    # Abrir el PDF
                    pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
                    
                    #función que separa cada examen por archivo
                    
                    lista_examenes = funcion_que_separa_cada_examen_por_archivo(pdf_document)

                    for examen in lista_examenes:
                        # Extraer contenido (asumiendo que tienes esta función)
                        content = extract_pdf_content(pdf_document,examen)
                        
                        # Agregar los datos al resultado
                        results.append({                        
                        # "test": content.get("test"),
                        "Archivo": obj.object_name.split('/')[-1],  # Solo el nombre del archivo
                        "Ruta Completa": obj.object_name,
                        "Folio": content.get("Folio"),
                        "Fecha Impresion": content.get("Fecha Impresion"),
                        "Identificacion": content.get("Identificacion"),
                        "Numero de Carpeta": content.get("Numero de carpeta"),
                        "Nombre del Paciente": content.get("Nombre del paciente"),
                        "Edad Actual": content.get("Edad Actual"),
                        "Sexo": content.get("Sexo"),
                        "Ingreso": content.get("Ingreso"),
                        "Fecha de Ingreso": content.get("Fecha de Ingreso"),
                        "Fecha Evolucion": content.get("Fecha Evolucion"),
                        "Cama": content.get("Cama"),
                        "Servicio": content.get("Servicio"),
                        "Entidad": content.get("Entidad"),
                        "Fecha de Toma": content.get("Fecha de Toma"),
                        "Fecha de Recepcion": content.get("Fecha de Recepcion"),
                        "Fecha de Informe": content.get("Fecha de Informe"),
                        "Informe de Patologia": content.get("Informe de Patologia"),
                        "Material Enviado": content.get("Material Enviado"),
                        "Descripcion Macroscopica": content.get("Descripcion Macroscopica"),
                        "Descripcion Microscopica": content.get("Descripcion Microscopica"),
                        "Diagnostico Histopatologico": content.get("Diagnostico Histopatologico"),
                        "Medico Patologo": content.get("Medico Patologo"),
                        "Registro Profesional": content.get("Registro Profesional"),
                        "Nombre reporte": content.get("Nombre reporte"),
                        "Usuario": content.get("Usuario")
                    })
                    
                except Exception as e:
                    print(f"Error procesando {obj.object_name}: {e}")
                finally:
                    if pdf_document is not None:
                        pdf_document.close()
                    if 'response' in locals():
                        response.close()
                        response.release_conn()
    
    except S3Error as e:
        print(f"Error al acceder al bucket {bucket_name}: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    
    # Convertir resultados en un DataFrame
    return pd.DataFrame(results)

In [ ]:
# Ejemplo de uso
df_resultados = process_all_pdfs_in_minio(
    minio_client=minio_client,  # Tu cliente MinIO ya conectado
    bucket_name="yachay-landing",
    folder_path="Laboratorios/4. HUDN"  # Ruta dentro del bucket
)

# Mostrar resultados
print(f"Se procesaron {len(df_resultados)} archivos PDF")
display(df_resultados.head())

Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q17-23-87245408.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q29-23-5378719.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q33-23-12954743.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q42-23-87471437.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q71-23-30708190.pdf
Se procesaron 5 archivos PDF


,Archivo,Ruta Completa,Numero de Informe,Nombre del Paciente,Identificacion,Edad,Telefono,Fecha de Toma de Muestra,Fecha de Ingreso,Fecha de Informe,Entidad,EPS,Servicio,Muestra Remitida,Descripcion Macroscopica,Descripcion Microscopica,Diagnostico,Comentario
0,Q17-23-87245408.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q17-23-...,Q17-23,FLORESMIRO REALPE NARVAEZ,CC 87245408,61,3117207145,5/01/2023,10/01/2023,19/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,IMAGENOLOGIA,BIOPSIA,se recibe dos muestras: 1. Rotulado “próstata ...,Estudio microscópico realizado. Ver diagnostic...,[1. CILINDROS DE TEJIDO PROSTATICO LADO DERECH...,None
1,Q29-23-5378719.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q29-23-...,Q29-23,PEDRO ALFONSO PORTILLO INSUASTI,CC 5378719,80,SIN,10/01/2023,10/01/2023,9/02/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,IMAGENOLOGIA,BIOPSIA,se recibe dos muestras: 1. Rotulado “biopsia p...,A y B) En los cortes histológicos se reconocen...,[1. LOBULO PROSTATICO DERECHO; BIOPSIA TRANSRE...,None
2,Q33-23-12954743.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q33-23-...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[REVISION E INMUNOHISTOQUIMICA DE UN BLOQUE DE...,None
3,Q42-23-87471437.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q42-23-...,Q42-23,SILVIO ORLANDO IMBAJOA SACANAMBUY,CC 87471437,55,3233645762,3/01/2023,11/01/2023,13/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,GASTROENTEROLOGIA,ULCERA,se recibe dos muestras: 1. Rotulado “ulcera gá...,A) En los cortes histológicos se observa mucos...,[1. BORDE DE ULCERA GASTRICA; BIOPSIA:  ADENO...,None
4,Q71-23-30708190.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q71-23-...,Q71-23,BLANCA MARLENI MONTILLA DE TORRES,CC 30708190,66,3164040760,13/01/2023,13/01/2023,23/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,DERMATOLOGIA,BIOPSIA,"Rotulado “piel de mejilla derecha”, en formol ...",Piel con hiperparaqueratosis. Epidermis irregu...,[PIEL MEJILLA DERECHA: BIOPSIA:  LESION LENTI...,None


In [6]:
ruta_fecha = generar_ruta_fecha()

In [ ]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_resultados,
    bucket_name="yachay-bronze",
    ruta_destino=f"laboratorios/{ruta_fecha}/laboratorio HU Departamental/laboratorio_hu_departamental",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/laboratorios/2025/05/06/laboratorio Jimenez Pasto/laboratorio_jimenes_pasto.csv
